In [1]:
import numpy
import random

In [2]:
#convertir cada solución de matriz a vector
def mat_to_vector(mat_pop_weights):
    pop_weights_vector = []
    for sol_idx in range(mat_pop_weights.shape[0]):
        curr_vector = []
        for layer_idx  in range(mat_pop_weights.shape[1]):
            vector_weights = numpy.reshape(mat_pop_weights[sol_idx, layer_idx],newshape=(mat_pop_weights[sol_idx, layer_idx].size))
            curr_vector.extend(vector_weights)
        pop_weights_vector.append(curr_vector)
        return numpy.array(pop_weights_vector)

In [3]:
#convertir cada solución de vector a matriz
def vector_to_mat(vector_pop_weights, mat_pop_weights):
    mat_weights = []
    for sol_idx in range(mat_pop_weights.shape[0]):
        start = 0
        end = 0
        for layer_idx in range(mat_pop_weights.shape[1]):
            end = end + mat_pop_weights[sol_idx, layer_idx].size
            curr_vector = vector_pop_weights[sol_idx, start:end]
            mat_layer_weights = numpy.reshape(curr_vector, newshape=(mat_pop_weights[sol_idx, layer_idx].shape))
            mat_weights.append(mat_layer_weights)
            start = end
    return numpy.reshape(mat_weights, newshape=mat_pop_weights.shape)

In [5]:
# seleccionar los mejores individuos de la generación, para la descendencia de la próxima generación
def select_mating_pool(pop, fitness, num_parents):
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

In [9]:
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    crossover_point = numpy.uint32(offspring_size[1]/2)
    
    for k in range(offspring_size[0]):
        # índice primer match padres
        parent1_idx = k%parents.shape[0]
        # índice segundo match padres
        parent2_idx = (k+1)%parents.shape[0]
        # primer descendencia, mitad genes del primer padre
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # primer descendencia, mitad genes del segundo padre
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [10]:
def mutation(offspring_crossover, mutation_percent):
    num_mutations = numpy.uint32((mutation_percent*offspring_crossover.shape[1])/100)
    mutation_indices = numpy.array(random.sample(range(0, offspring_crossover.shape[1]), num_mutations))
    # Mutación cambia un gen único en cada descendencia al azar
    for idx in range(offspring_crossover.shape[0]):
        # el valor aleatorio es añadido al gen
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, mutation_indices] = offspring_crossover[idx, mutation_indices] + random_value
    return offspring_crossover